# Predicting Memorability


In this notebook I have combined the videos captions, C3D and HMP features to predict video memorability and run them through an Gradient Boosting regressor model after experimenting with different models and combinations of features.

The notebook is divided into two sections

1. Final ML Model - The final model which I used for the prediction task i.e. Gradient Boosting regressor model with a combination of captions, C3D and HMP.
2. Explorations - Different models were run with individual features and combination of features to find the best model for prediction.

## Installing and setting up Libraries

In [1]:
!pip install pyprind

In [2]:
!pip install xgboost 

In [3]:
#import panda and other libraries required

import os
import pandas as pd
from keras import Sequential
from keras import layers
from keras import regularizers
import numpy as np
from string import punctuation
import pyprind
from collections import Counter
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras import preprocessing
from functools import reduce
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from numpy import loadtxt   
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn import ensemble

#Randomising for reproducability
from numpy.random import seed
seed(1)
import tensorflow 
tensorflow.random.set_random_seed(1)

Using TensorFlow backend.
D:\Program\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Program\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Program\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Program\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a s

#### Spearman Function

In [4]:
def Get_score(Y_pred,Y_true):    #Spearman function
    '''Calculate the Spearmann"s correlation coefficient'''
    Y_pred = np.squeeze(Y_pred)
    Y_true = np.squeeze(Y_true)
    if Y_pred.shape != Y_true.shape:
        print('Input shapes don\'t match!')
    else:
        if len(Y_pred.shape) == 1:
            Res = pd.DataFrame({'Y_true':Y_true,'Y_pred':Y_pred})
            score_mat = Res[['Y_true','Y_pred']].corr(method='spearman',min_periods=1)
            print('The Spearman\'s correlation coefficient is: %.3f' % score_mat.iloc[1][0])
        else:
            for ii in range(Y_pred.shape[1]):
                Get_score(Y_pred[:,ii],Y_true[:,ii])

In [5]:
print(tensorflow.__version__)

1.14.0


## Gradient Boosting Regressor (Final model) is run on a combination of the features (Captions + C3D + HMP)

#### Loading Caption features Test Set

In [105]:
# load the test captions txt file
test_cap_path = 'D:/MCM_Second_Sem/Machine Learning/CA684_Assignment/Test-set/Captions_test/test-set-1_video-captions.txt'
df_test_cap=read_caps(test_cap_path)# load the test captions txt file

In [106]:
df_test_cap.head()

,video,caption
0,video7494.webm,green-jeep-struggling-to-drive-over-huge-rocks
1,video7495.webm,hiking-woman-tourist-is-walking-forward-in-mou...
2,video7496.webm,close-up-of-african-american-doctors-hands-usi...
3,video7497.webm,slow-motion-of-a-man-using-treadmill-in-the-gy...
4,video7498.webm,slow-motion-of-photographer-in-national-park


#### Loading C3D features Test Set

In [107]:
# Load testing data - this should load in test set of video features 

vid = df_test_cap.video.values  #pull in video labels first

Features_test = pd.DataFrame({'video': vid,
                   'C3D': [read_C3D('D:/MCM_Second_Sem/Machine Learning/CA684_Assignment/Test-set/C3D_test/'+os.path.splitext(item)[0]+'.txt') for item in vid],
                       })

In [108]:
len(Features_test)

2000

In [109]:
Features_test.head()

,video,C3D
0,video7494.webm,"[0.00013576, 0.0002133, 0.00124759, 1.149e-05,..."
1,video7495.webm,"[0.00012659, 2.377e-05, 0.04950115, 2.418e-05,..."
2,video7496.webm,"[0.48898152, 0.00058536, 2.454e-05, 0.00016031..."
3,video7497.webm,"[0.24925078, 0.00101763, 8.02e-06, 1.92e-05, 2..."
4,video7498.webm,"[0.00989415, 3.413e-05, 0.00261261, 2.1e-06, 1..."


#### Loading HMP features Test Set

In [110]:
# Load testing data - this should load in test set of video features

vid = df_test_cap.video.values  #pull in video labels first

HMP_Features_test = pd.DataFrame({'video': vid,
                   'HMP': [read_HMP('D:/MCM_Second_Sem/Machine Learning/CA684_Assignment/Test-set/HMP_test/'+os.path.splitext(item)[0]+'.txt') for item in vid],
                       })

In [111]:
len(HMP_Features_test)

2000

In [112]:
HMP_Features_test.head()

,video,HMP
0,video7494.webm,"[0.022039, 0.005264, 9.8e-05, 0.0, 0.003689, 0..."
1,video7495.webm,"[0.004694, 0.001934, 4e-05, 0.0, 0.001226, 5.8..."
2,video7496.webm,"[0.02653, 0.006208, 5.8e-05, 0.0, 0.00228, 3.1..."
3,video7497.webm,"[0.045109, 0.008317, 0.000151, 0.0, 0.005598, ..."
4,video7498.webm,"[0.034691, 0.008619, 0.000412, 0.0, 0.028355, ..."


#### Preprocessing Captions

In [113]:
counts = Counter()
# setup prograss tracker
pbar = pyprind.ProgBar(len(df_test_cap['caption']), title='Counting word occurrences')
for i, cap in enumerate(df_test_cap['caption']):
    # replace punctuations with space
    # convert words to lower case 
    text = ''.join([c if c not in punctuation else ' ' for c in cap]).lower()
    df_test_cap.loc[i,'caption'] = text
    pbar.update()
    counts.update(text.split())

Counting word occurrences
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


In [114]:
# build the word index
len_token = len(counts)
tokenizer = Tokenizer(num_words=len_token)
print(len_token)

3197


In [115]:
tokenizer.fit_on_texts(list(df_test_cap.caption.values))

In [116]:
one_hot_res = tokenizer.texts_to_matrix(list(df_test_cap.caption.values),mode='binary')
test_sequences = tokenizer.texts_to_sequences(list(df_test_cap.caption.values))

In [117]:
X_test_seq = np.zeros((len(test_sequences),max_len))
for i in range(len(test_sequences)):
    n = len(test_sequences[i])
    if n==0:
        print(i)
    else:
        X_test_seq[i,-n:] = test_sequences[i]
X_test_seq.shape

(2000, 50)

In [118]:
df_test_seq = pd.DataFrame(X_test_seq)    #create dataframe with the sequences
df_test_seq = pd.concat([df_test_cap, df_test_seq], axis = 1)   #merging sequnces dataframe and the captions dataframe to I can line up video name to each sequence
df_test_seq = df_test_seq.drop("caption", axis=1)  #removing column with word version of caption so we have a dataframe where the integer sequences are algined with the primary key (video names)

In [119]:
df_test_seq.head()

,video,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,video7494.webm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,61.0,640.0,1088.0,12.0,1089.0,66.0,428.0,283.0
1,video7495.webm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,27.0,1605.0,118.0,12.0,1090.0,17.0,6.0,27.0,13.0,14.0
2,video7496.webm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,90.0,362.0,53.0,29.0,5.0,1606.0,12.0,1091.0,1607.0,1092.0
3,video7497.webm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1093.0,1608.0,804.0,26.0,1609.0,35.0,34.0,7.0,805.0,1610.0
4,video7498.webm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,13.0,14.0,7.0,1611.0,1.0,76.0,19.0


#### Preprocessing C3D features

In [120]:
Features_test1 = pd.DataFrame(Features_test.C3D.apply(pd.Series)) 
Features_test2 = pd.concat([Features_test, Features_test1], axis = 1)  #merges features and features1 (splits out into columns)
Features_test2 = Features_test2.drop("C3D", axis=1)    #drops column where all c3d features are combined in 1 column

In [121]:
Features_test2.head()

,video,0,1,2,3,4,5,6,7,8,...,91,92,93,94,95,96,97,98,99,100
0,video7494.webm,0.000136,0.000213,0.001248,0.000011,1.180000e-06,4.276000e-05,5.800000e-07,5.820000e-06,0.000007,...,0.000003,5.620000e-06,1.260000e-06,0.000199,1.500000e-06,2.000000e-07,5.840000e-06,0.000006,0.000210,0.195120
1,video7495.webm,0.000127,0.000024,0.049501,0.000024,2.361000e-05,3.440000e-06,5.102000e-05,1.240200e-04,0.000023,...,0.000173,3.019200e-04,2.649800e-04,0.000163,1.050000e-05,2.540000e-06,3.619300e-04,0.000016,0.000010,0.000392
2,video7496.webm,0.488982,0.000585,0.000025,0.000160,2.400000e-07,8.300000e-07,3.620000e-06,5.200000e-07,0.000027,...,0.000002,1.900000e-07,3.200000e-07,0.001802,2.640000e-06,9.000000e-08,2.600000e-07,0.000067,0.000123,0.000015
3,video7497.webm,0.249251,0.001018,0.000008,0.000019,2.200000e-07,8.800000e-07,1.646560e-02,4.701000e-05,0.000025,...,0.000065,8.400000e-07,1.700000e-07,0.000808,9.700000e-07,7.670000e-06,9.100000e-07,0.000015,0.000707,0.000010
4,video7498.webm,0.009894,0.000034,0.002613,0.000002,1.010000e-06,8.770000e-06,2.610000e-06,4.500000e-07,0.000005,...,0.000012,6.600000e-07,8.770000e-06,0.000132,7.400000e-07,2.120000e-06,8.260000e-06,0.001402,0.000745,0.000456


#### Preprocessing HMP features

In [122]:
df_hmp_test1 = pd.DataFrame(HMP_Features_test.HMP.apply(pd.Series)) 
df_hmp_test = pd.concat([HMP_Features_test, df_hmp_test1], axis = 1)  #merges features and features1 (splits out into columns)
df_hmp_test = df_hmp_test.drop("HMP", axis=1)    #drops column where all c3d features are combined in 1 column
df_hmp_test.head()

,video,0,1,2,3,4,5,6,7,8,...,6065,6066,6067,6068,6069,6070,6071,6072,6073,6074
0,video7494.webm,0.022039,0.005264,0.000098,0.0,0.003689,0.000129,0.0,0.000000,0.000045,...,0.000025,0.000114,0.000056,0.000002,0.000136,0.000388,0.000038,0.000165,0.000488,0.000187
1,video7495.webm,0.004694,0.001934,0.000040,0.0,0.001226,0.000058,0.0,0.000000,0.000025,...,0.000000,0.000056,0.000027,0.000002,0.000087,0.000121,0.000004,0.000109,0.000136,0.000029
2,video7496.webm,0.026530,0.006208,0.000058,0.0,0.002280,0.000031,0.0,0.000000,0.000009,...,0.000011,0.000034,0.000011,0.000002,0.000063,0.000081,0.000004,0.000092,0.000143,0.000051
3,video7497.webm,0.045109,0.008317,0.000151,0.0,0.005598,0.000128,0.0,0.000008,0.000050,...,0.000035,0.000256,0.000083,0.000003,0.000116,0.000467,0.000020,0.000279,0.000470,0.000219
4,video7498.webm,0.034691,0.008619,0.000412,0.0,0.028355,0.000538,0.0,0.000000,0.000325,...,0.000009,0.000148,0.000038,0.000002,0.000117,0.000206,0.000022,0.000228,0.000334,0.000086


Merging the features

In [124]:
data_frames = [df_test_seq, Features_test2, df_hmp_test]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['video'],
                                            how='outer'), data_frames)

In [127]:
df_test_x = df_merged.drop('video', axis = 1)

In [128]:
df_test_x.head()

,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,6065,6066,6067,6068,6069,6070,6071,6072,6073,6074
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000025,0.000114,0.000056,0.000002,0.000136,0.000388,0.000038,0.000165,0.000488,0.000187
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000056,0.000027,0.000002,0.000087,0.000121,0.000004,0.000109,0.000136,0.000029
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000011,0.000034,0.000011,0.000002,0.000063,0.000081,0.000004,0.000092,0.000143,0.000051
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000035,0.000256,0.000083,0.000003,0.000116,0.000467,0.000020,0.000279,0.000470,0.000219
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000009,0.000148,0.000038,0.000002,0.000117,0.000206,0.000022,0.000228,0.000334,0.000086


In [187]:
df_test_x.shape

(2000, 6226)

#### Splitting into Long term and Short term

In [188]:
X_full_train = df_x.values
Y_full_train_st = labels['short-term_memorability'].values    
Y_full_train_lt = labels['long-term_memorability'].values

In [189]:
print('X_train', X_full_train.shape)
print('Y_train lt', Y_full_train_lt.shape)
print('Y_train st', Y_full_train_st.shape)

X_train (6000, 6226)
Y_train lt (6000,)
Y_train st (6000,)


In [190]:
#Model Params
params = {'n_estimators':650, 'max_depth':12, 'min_samples_split':2, 'learning_rate':0.01, 'loss':'lad'}
clf_final_st = ensemble.GradientBoostingRegressor(**params)

In [191]:
#fit to training set short term
clf_final_st.fit(X_full_train, Y_full_train_st)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.01, loss='lad',
                          max_depth=12, max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=650,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [192]:
prediction_st = clf_final_st.predict(df_test_x)
print('long term:')
print(prediction_st)

long term:
[0.85388672 0.82191784 0.87906372 ... 0.88205003 0.83946111 0.88370721]


In [193]:
#Model Params
params = {'n_estimators':650, 'max_depth':12, 'min_samples_split':2, 'learning_rate':0.01, 'loss':'lad'}
clf_final_lt = ensemble.GradientBoostingRegressor(**params)

In [194]:
#fit to training set long term
clf_final_lt.fit(X_full_train, Y_full_train_lt)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.01, loss='lad',
                          max_depth=12, max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=650,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [195]:
#predict test set long term
prediction_lt = clf_final_lt.predict(df_test_x)
print('short term:')
print(prediction_lt)

short term:
[0.78669624 0.77894768 0.77157225 ... 0.81371258 0.77749282 0.80482015]


In [196]:
#Creating data frames for both st and lt predictions
df_predictions_final_st = pd.DataFrame(prediction_st)
df_predictions_final_st.columns = ['short-term_memorability']
df_predictions_final_lt = pd.DataFrame(prediction_lt)
df_predictions_final_lt.columns = ['long-term_memorability']

In [197]:
print(df_predictions_final_st)

      short-term_memorability
0                    0.853887
1                    0.821918
2                    0.879064
3                    0.908420
4                    0.770162
...                       ...
1995                 0.860781
1996                 0.893002
1997                 0.882050
1998                 0.839461
1999                 0.883707

[2000 rows x 1 columns]


In [ ]:
print(df_predictions_final_lt)

In [198]:
my_predictions_st = pd.concat([df_test_cap, df_predictions_final_st], axis = 1)  #merges predictions st to captions
my_predictions_st = my_predictions_st.drop("caption", axis=1)    #drop captions row

In [199]:
print(my_predictions_st)

                video  short-term_memorability
0      video7494.webm                 0.853887
1      video7495.webm                 0.821918
2      video7496.webm                 0.879064
3      video7497.webm                 0.908420
4      video7498.webm                 0.770162
...               ...                      ...
1995  video10004.webm                 0.860781
1996  video10005.webm                 0.893002
1997  video10006.webm                 0.882050
1998  video10007.webm                 0.839461
1999  video10008.webm                 0.883707

[2000 rows x 2 columns]


In [200]:
my_predictions_lt = pd.concat([df_test_cap, df_predictions_final_lt], axis = 1)  #merges predictions st to captions
my_predictions_lt = my_predictions_lt.drop("caption", axis=1)    #drop captions row

In [201]:
print(my_predictions_lt)

                video  long-term_memorability
0      video7494.webm                0.786696
1      video7495.webm                0.778948
2      video7496.webm                0.771572
3      video7497.webm                0.796987
4      video7498.webm                0.753085
...               ...                     ...
1995  video10004.webm                0.764350
1996  video10005.webm                0.799399
1997  video10006.webm                0.813713
1998  video10007.webm                0.777493
1999  video10008.webm                0.804820

[2000 rows x 2 columns]


In [202]:
df_predictions_final = pd.merge(my_predictions_st, my_predictions_lt, on='video')
df_predictions_final = df_predictions_final.drop("video", axis=1)

In [203]:
print(df_predictions_final)    #Here is the data frame containing my short term and long term prediction having dropped the video column

      short-term_memorability  long-term_memorability
0                    0.853887                0.786696
1                    0.821918                0.778948
2                    0.879064                0.771572
3                    0.908420                0.796987
4                    0.770162                0.753085
...                       ...                     ...
1995                 0.860781                0.764350
1996                 0.893002                0.799399
1997                 0.882050                0.813713
1998                 0.839461                0.777493
1999                 0.883707                0.804820

[2000 rows x 2 columns]


In [204]:
template=pd.read_csv('D:/MCM_Second_Sem/Machine Learning/CA684_Assignment/ground_truth_template.csv')  #pulls in csv file from google drive and store in template
template1 = pd.DataFrame(template)  #create data frame

In [205]:
print(template1)

      video  short-term_memorability  nb_short-term_annotations  \
0      7494                      NaN                         33   
1      7495                      NaN                         34   
2      7496                      NaN                         32   
3      7497                      NaN                         33   
4      7498                      NaN                         33   
...     ...                      ...                        ...   
1995  10004                      NaN                         34   
1996  10005                      NaN                         34   
1997  10006                      NaN                         34   
1998  10007                      NaN                         34   
1999  10008                      NaN                         33   

      long-term_memorability  nb_long-term_annotations  
0                        NaN                        12  
1                        NaN                        10  
2                       

In [206]:
ground_truth_values = template1.drop('short-term_memorability', axis=1)   #drop short term memorability row
ground_truth_values = ground_truth_values.drop('long-term_memorability', axis=1)   #drop long term memorability row


ground_truth_values = pd.concat([ground_truth_values, df_predictions_final], axis = 1)   #merge with my final predictions

ground_truth_values = ground_truth_values[['video', 'short-term_memorability','nb_short-term_annotations','long-term_memorability', 'nb_long-term_annotations']]  #rearrange columns to right order

In [207]:
print(ground_truth_values)

      video  short-term_memorability  nb_short-term_annotations  \
0      7494                 0.853887                         33   
1      7495                 0.821918                         34   
2      7496                 0.879064                         32   
3      7497                 0.908420                         33   
4      7498                 0.770162                         33   
...     ...                      ...                        ...   
1995  10004                 0.860781                         34   
1996  10005                 0.893002                         34   
1997  10006                 0.882050                         34   
1998  10007                 0.839461                         34   
1999  10008                 0.883707                         33   

      long-term_memorability  nb_long-term_annotations  
0                   0.786696                        12  
1                   0.778948                        10  
2                   0.77

In [ ]:
ground_truth_values.to_csv('ground_truth_values.csv')

# Exploration

## Loading and preprocessing Features

### 1. Captions

In [6]:
# load labels and captions 
def read_caps(fname):
    """Load the captions into a dataframe"""
    vn = []
    cap = []
    df = pd.DataFrame();
    with open(fname) as f:
        for line in f:
            pairs = line.split()
            vn.append(pairs[0])
            cap.append(pairs[1])
        df['video']=vn
        df['caption']=cap
    return df


# load the captions txt file
cap_path = 'D:\MCM_Second_Sem\Machine Learning\CA684_Assignment\Dev-set\Captions\dev-set_video-captions.txt'
df_cap=read_caps(cap_path)

labels=pd.read_csv('D:\MCM_Second_Sem\Machine Learning\CA684_Assignment\Dev-set\Ground-truth\ground-truth.csv')

In [7]:
counts = Counter()
# setup prograss tracker
pbar = pyprind.ProgBar(len(df_cap['caption']), title='Counting word occurrences')
for i, cap in enumerate(df_cap['caption']):
    # replace punctuations with space
    # convert words to lower case 
    text = ''.join([c if c not in punctuation else ' ' for c in cap]).lower()
    df_cap.loc[i,'caption'] = text
    pbar.update()
    counts.update(text.split())

Counting word occurrences
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


In [8]:
# build the word index
len_token = len(counts)
tokenizer = Tokenizer(num_words=len_token)
print(len_token)

5191


In [9]:
tokenizer.fit_on_texts(list(df_cap.caption.values)) #fit a list of captions to the tokenizer
#the tokenizer vectorizes a text corpus, by turning each text into either a sequence of integers

In [10]:
one_hot_res = tokenizer.texts_to_matrix(list(df_cap.caption.values),mode='binary')
sequences = tokenizer.texts_to_sequences(list(df_cap.caption.values))

In [11]:
# calculating max length
max_len = 50

In [12]:
X_seq = np.zeros((len(sequences),max_len))  #creating sequences of tokenized captions (6000 rows of 50 columns, padded with zeros when cpations are not of length 50)
for i in range(len(sequences)):
    n = len(sequences[i])
    if n==0:
        print(i)
    else:
        X_seq[i,-n:] = sequences[i]
X_seq.shape

(6000, 50)

In [13]:
df_seq = pd.DataFrame(X_seq)    #create dataframe with the sequences
df_seq = pd.concat([df_cap, df_seq], axis = 1)   #merging sequnces dataframe and the captions dataframe to I can line up video name to each sequence
df_seq = df_seq.drop("caption", axis=1)  #removing column with word version of caption so we have a dataframe where the integer sequences are algined with the primary key (video names)

In [14]:
df_seq.head()  #final dataframe for captions

,video,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,video3.webm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,724.0,7.0,35.0,884.0,384.0,91.0
1,video4.webm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2272.0,85.0,259.0,6.0,213.0,1.0,89.0,27.0
2,video6.webm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,131.0,312.0
3,video8.webm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,725.0,993.0,3098.0,9.0,165.0
4,video10.webm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,15.0,118.0,3.0,515.0,356.0,27.0


### 2. HMP features

In [15]:
def read_HMP(fname):
    """Scan HMP(Histogram of Motion Patterns) features from file"""
    with open(fname) as f:
        for line in f:
            pairs=line.split()
            HMP_temp = { int(p.split(':')[0]) : float(p.split(':')[1]) for p in pairs}
    # there are 6075 bins, fill zeros
    HMP = np.zeros(6075)
    for idx in HMP_temp.keys():
        HMP[idx-1] = HMP_temp[idx]            
    return HMP

In [16]:
vid = labels.video.values  #pull in video labels first

hmp_features = pd.DataFrame({'video': vid,
                   'HMP': [read_HMP('D:/MCM_Second_Sem/Machine Learning/CA684_Assignment/Dev-set/HMP/'+os.path.splitext(item)[0]+'.txt') for item in vid],
                       })

In [17]:
hmp_features.head()

,video,HMP
0,video3.webm,"[0.125563, 0.024036, 0.000314, 0.0, 0.015864, ..."
1,video4.webm,"[0.007526, 0.001421, 6.8e-05, 0.0, 0.001184, 0..."
2,video6.webm,"[0.109584, 0.018978, 0.000289, 0.0, 0.008774, ..."
3,video8.webm,"[0.120431, 0.013561, 0.000277, 0.0, 0.018974, ..."
4,video10.webm,"[0.005026, 0.001356, 5.5e-05, 0.0, 0.000665, 2..."


In [18]:
feat1 = pd.DataFrame(hmp_features.HMP.apply(pd.Series))  #split out hmp column into individual rows
feat2 = pd.concat([hmp_features, feat1], axis = 1)  #merges feat1 and hmp_features (splits out into columns)
feat2 = feat2.drop("HMP", axis=1) 

In [19]:
feat2.head()

,video,0,1,2,3,4,5,6,7,8,...,6065,6066,6067,6068,6069,6070,6071,6072,6073,6074
0,video3.webm,0.125563,0.024036,0.000314,0.0,0.015864,0.000358,0.0,0.000000,0.000086,...,0.000000,0.000393,0.000279,0.000000,0.000289,0.001926,0.000000,0.000086,0.000580,0.000000
1,video4.webm,0.007526,0.001421,0.000068,0.0,0.001184,0.000143,0.0,0.000000,0.000079,...,0.000053,0.000244,0.000066,0.000000,0.000081,0.000617,0.000094,0.000220,0.000762,0.001224
2,video6.webm,0.109584,0.018978,0.000289,0.0,0.008774,0.000208,0.0,0.000002,0.000088,...,0.000007,0.000054,0.000045,0.000000,0.000028,0.000291,0.000033,0.000052,0.000258,0.000215
3,video8.webm,0.120431,0.013561,0.000277,0.0,0.018974,0.000913,0.0,0.000024,0.000713,...,0.000059,0.001110,0.000075,0.000008,0.000333,0.000793,0.000101,0.000588,0.000503,0.000452
4,video10.webm,0.005026,0.001356,0.000055,0.0,0.000665,0.000029,0.0,0.000000,0.000024,...,0.000009,0.000882,0.000200,0.000009,0.000559,0.001097,0.000018,0.000632,0.001128,0.000064


### 3. C3D Features

In [20]:
def read_C3D(fname):   #function to read c3d vectors into dataframe
    with open(fname) as f:
        for line in f:
            C3D =[float(item) for item in line.split()] # convert to float type, using default separator
    return C3D

def vname2ID(vnames):  #function to read video name and format correclty for dataframe, which will be used as primary key to merge sequences dataframe and c3d dataframe
    vid = [ os.path.splitext(vn)[0]+'.webm' for vn in vnames]
    return vid

In [21]:
# Load testing data - this should load in test set of videos (again this can take around 10 mins to run)

vid = labels.video.values  #pull in video labels first

c3d_features = pd.DataFrame({'video': vid,
                   'C3D': [read_C3D('D:/MCM_Second_Sem/Machine Learning/CA684_Assignment/Dev-set/C3D/'+os.path.splitext(item)[0]+'.txt') for item in vid],
                       })

In [22]:
c3d_features.head()

,video,C3D
0,video3.webm,"[0.02024942, 0.0015778, 0.00082625, 0.00094509..."
1,video4.webm,"[0.000118, 0.00089075, 0.00018769, 4.543e-05, ..."
2,video6.webm,"[0.01176522, 0.00074577, 0.00078353, 1.328e-05..."
3,video8.webm,"[0.00022343, 0.00016499, 7.35e-06, 1.615e-05, ..."
4,video10.webm,"[9.006e-05, 0.00061494, 0.00343634, 0.00128092..."


In [23]:
features1 = pd.DataFrame(c3d_features.C3D.apply(pd.Series))  #split out c3d column into individual rows
features2 = pd.concat([c3d_features, features1], axis = 1)  #merges c3d_features and features1 (splits out into columns)
features2 = features2.drop("C3D", axis=1) 

In [24]:
features2.head()

,video,0,1,2,3,4,5,6,7,8,...,91,92,93,94,95,96,97,98,99,100
0,video3.webm,0.020249,0.001578,0.000826,0.000945,0.000063,0.000003,0.001162,0.000097,0.000022,...,0.001042,0.000161,0.000257,0.046617,0.000156,0.000006,0.000537,0.000339,0.008437,0.000470
1,video4.webm,0.000118,0.000891,0.000188,0.000045,0.000063,0.000002,0.000641,0.000011,0.000017,...,0.000582,0.000393,0.000864,0.000947,0.000136,0.000007,0.000360,0.000159,0.001025,0.000020
2,video6.webm,0.011765,0.000746,0.000784,0.000013,0.000007,0.000028,0.000041,0.000014,0.000041,...,0.000224,0.000003,0.000031,0.002538,0.000104,0.000005,0.000064,0.005380,0.001027,0.001384
3,video8.webm,0.000223,0.000165,0.000007,0.000016,0.000005,0.000014,0.000154,0.000008,0.000004,...,0.000046,0.000009,0.000023,0.000053,0.000048,0.000019,0.000001,0.000004,0.000380,0.000029
4,video10.webm,0.000090,0.000615,0.003436,0.001281,0.003551,0.000313,0.000042,0.000018,0.000157,...,0.000037,0.000690,0.000171,0.000231,0.000637,0.000040,0.000061,0.000075,0.000002,0.001323


#### Merge Features

In [25]:
df_x2 = pd.merge(df_seq, features2, on='video') #captions + C3D

In [26]:
data_frames = [df_seq, features2, feat2]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['video'],
                                            how='outer'), data_frames)

In [27]:
df_merged.head()

,video,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,...,6065,6066,6067,6068,6069,6070,6071,6072,6073,6074
0,video3.webm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000393,0.000279,0.000000,0.000289,0.001926,0.000000,0.000086,0.000580,0.000000
1,video4.webm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000053,0.000244,0.000066,0.000000,0.000081,0.000617,0.000094,0.000220,0.000762,0.001224
2,video6.webm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000007,0.000054,0.000045,0.000000,0.000028,0.000291,0.000033,0.000052,0.000258,0.000215
3,video8.webm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000059,0.001110,0.000075,0.000008,0.000333,0.000793,0.000101,0.000588,0.000503,0.000452
4,video10.webm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000009,0.000882,0.000200,0.000009,0.000559,0.001097,0.000018,0.000632,0.001128,0.000064


In [28]:
df_x = df_merged.drop('video', axis = 1)  #drop video column

In [30]:
df_x.head()

,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,6065,6066,6067,6068,6069,6070,6071,6072,6073,6074
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000393,0.000279,0.000000,0.000289,0.001926,0.000000,0.000086,0.000580,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000053,0.000244,0.000066,0.000000,0.000081,0.000617,0.000094,0.000220,0.000762,0.001224
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000007,0.000054,0.000045,0.000000,0.000028,0.000291,0.000033,0.000052,0.000258,0.000215
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000059,0.001110,0.000075,0.000008,0.000333,0.000793,0.000101,0.000588,0.000503,0.000452
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000009,0.000882,0.000200,0.000009,0.000559,0.001097,0.000018,0.000632,0.001128,0.000064


# Applying features to models to find best model

### Running models on Captions

In [29]:
df_seq=df_seq.drop('video', axis = 1)

In [76]:
Y_st = labels['short-term_memorability'].values # st targets
Y_lt = labels['long-term_memorability'].values  # lt targets
X = df_seq.values # sequences & C3D merged

In [77]:
X_train_st, X_test_st, Y_train_st, Y_test_st = train_test_split(X,Y_st, test_size=0.2, random_state=42) 
#splitting the short term dev data into a train and validate split of 80 to 20 with a random state for reproducability

X_train_lt, X_test_lt, Y_train_lt, Y_test_lt = train_test_split(X,Y_lt, test_size=0.2, random_state=42) 
# #splitting the long term dev data into a train and validate split of 80 to 20 with a random state for reproducability

In [78]:
#Just testing to see shape of data split for Short Term
print('X_train', X_train_st.shape)
print('X_test', X_test_st.shape)
print('Y_train', Y_train_st.shape)
print('Y_test', Y_test_st.shape)

X_train (4800, 50)
X_test (1200, 50)
Y_train (4800,)
Y_test (1200,)


In [79]:
#Just testing to see shape of data split for Long Term
print('X_train', X_train_lt.shape)
print('X_test', X_test_lt.shape)
print('Y_train', Y_train_lt.shape)
print('Y_test', Y_test_lt.shape)

X_train (4800, 50)
X_test (1200, 50)
Y_train (4800,)
Y_test (1200,)


#### Linear Regressor Model on Captions

In [36]:
LMregressor = LinearRegression()
LMregressor.fit(X_train_st, Y_train_st)
y_pred = LMregressor.predict(X_test_st)
print('Linear Regressor Short Term:')
Get_score(y_pred, Y_test_st)

Linear Regressor Short Term:
The Spearman's correlation coefficient is: -0.003


In [37]:
LMregressor = LinearRegression()
LMregressor.fit(X_train_lt, Y_train_lt)
y_pred = LMregressor.predict(X_test_lt)
print('Linear Regressor Long Term:')
Get_score(y_pred, Y_test_lt)

Linear Regressor Long Term:
The Spearman's correlation coefficient is: -0.042


#### Decision Tree Regressor Model on Captions

In [38]:
DTregressor = DecisionTreeRegressor()
DTregressor.fit(X_train_st, Y_train_st)
y_pred = DTregressor.predict(X_test_st)
print('Decision Tree Regressor Short Term:')
Get_score(y_pred, Y_test_st)

Decision Tree Regressor Short Term:
The Spearman's correlation coefficient is: 0.143


In [39]:
DTregressor = DecisionTreeRegressor()
DTregressor.fit(X_train_lt, Y_train_lt)
y_pred = DTregressor.predict(X_test_lt)
print('Decision Tree Regressor Long Term:')
Get_score(y_pred, Y_test_lt)

Decision Tree Regressor Long Term:
The Spearman's correlation coefficient is: 0.058


#### Random Forest Regressor Model on Captions

In [161]:
RFregressor = RandomForestRegressor(n_jobs=-1,n_estimators=100)
RFregressor.fit(X_train_st, Y_train_st)
y_pred = RFregressor.predict(X_test_st)
print('Random Forest Regressor Short Term:')
Get_score(y_pred, Y_test_st)

Random Forest Regressor Short Term:
The Spearman's correlation coefficient is: 0.254


In [162]:
RFregressor = RandomForestRegressor(n_jobs=-1,n_estimators=100)
RFregressor.fit(X_train_lt, Y_train_lt)
y_pred = RFregressor.predict(X_test_lt)
print('Random Forest Regressor Long Term:')
Get_score(y_pred, Y_test_lt)

Random Forest Regressor Long Term:
The Spearman's correlation coefficient is: 0.083


#### XGBoost Regressor Model on Captions

In [80]:
xg_reg = xgb.XGBRegressor(nthread=-1, learning_rate = 1, max_depth = 5, alpha = 10, n_estimators = 100, objective ='reg:squarederror')
xg_reg.fit(X_train_st,Y_train_st)
preds_y = xg_reg.predict(X_test_st)
print('XG Boosting Regressor Short Term:')
Get_score(preds_y, Y_test_st)

XG Boosting Regressor Short Term:
The Spearman's correlation coefficient is: 0.246


In [81]:
xg_reg = xgb.XGBRegressor(nthread=-1, learning_rate = 1, max_depth = 5, alpha = 10, n_estimators = 100, objective ='reg:squarederror')
xg_reg.fit(X_train_lt,Y_train_lt)
preds_y = xg_reg.predict(X_test_lt)
print('XG Boosting Regressor Long Term:')
Get_score(preds_y, Y_test_lt)

XG Boosting Regressor Long Term:
The Spearman's correlation coefficient is: 0.132


#### Gradient Boosting Regressor Model on Captions

In [45]:
clf = ensemble.GradientBoostingRegressor(n_estimators=650, max_depth=12, min_samples_split=2, learning_rate=0.01, loss='lad')

In [46]:
clf.fit(X_train_st, Y_train_st)
print('Gradient Boosting Regressor Short Term:')
print(Get_score(clf.predict(X_test_st), Y_test_st))

Gradient Boosting Regressor Short Term:
The Spearman's correlation coefficient is: 0.250
None


In [47]:
clf.fit(X_train_lt, Y_train_lt)
print('Gradient Boosting Regressor Long Term:')
print(Get_score(clf.predict(X_test_lt), Y_test_lt))

Gradient Boosting Regressor Long Term:
The Spearman's correlation coefficient is: 0.075
None


### Running models on C3D

In [36]:
features2=features2.drop('video', axis = 1)

In [82]:
Y_st = labels['short-term_memorability'].values # st targets
Y_lt = labels['long-term_memorability'].values  # lt targets
X = features2.values # sequences & C3D merged

In [83]:
X_train_st, X_test_st, Y_train_st, Y_test_st = train_test_split(X,Y_st, test_size=0.2, random_state=42) 
#splitting the short term dev data into a train and validate split of 80 to 20 with a random state for reproducability

X_train_lt, X_test_lt, Y_train_lt, Y_test_lt = train_test_split(X,Y_lt, test_size=0.2, random_state=42) 
# #splitting the long term dev data into a train and validate split of 80 to 20 with a random state for reproducability

In [84]:
#Just testing to see shape of data split for Short Term
print('X_train', X_train_st.shape)
print('X_test', X_test_st.shape)
print('Y_train', Y_train_st.shape)
print('Y_test', Y_test_st.shape)

X_train (4800, 101)
X_test (1200, 101)
Y_train (4800,)
Y_test (1200,)


In [85]:
#Just testing to see shape of data split for Long Term
print('X_train', X_train_lt.shape)
print('X_test', X_test_lt.shape)
print('Y_train', Y_train_lt.shape)
print('Y_test', Y_test_lt.shape)

X_train (4800, 101)
X_test (1200, 101)
Y_train (4800,)
Y_test (1200,)


#### Linear Regressor Model on C3D

In [53]:
LMregressor = LinearRegression()
LMregressor.fit(X_train_st, Y_train_st)
y_pred = LMregressor.predict(X_test_st)
print('Linear Regressor Short Term:')
Get_score(y_pred, Y_test_st)

Linear Regressor Short Term:
The Spearman's correlation coefficient is: 0.277


In [54]:
LMregressor = LinearRegression()
LMregressor.fit(X_train_lt, Y_train_lt)
y_pred = LMregressor.predict(X_test_lt)
print('Linear Regressor Long Term:')
Get_score(y_pred, Y_test_lt)

Linear Regressor Long Term:
The Spearman's correlation coefficient is: 0.103


#### Decision Tree Regressor Model  on C3D

In [55]:
DTregressor = DecisionTreeRegressor()
DTregressor.fit(X_train_st, Y_train_st)
y_pred = DTregressor.predict(X_test_st)
print('Decision Tree Regressor Short Term:')
Get_score(y_pred, Y_test_st)

Decision Tree Regressor Short Term:
The Spearman's correlation coefficient is: 0.124


In [56]:
DTregressor = DecisionTreeRegressor()
DTregressor.fit(X_train_lt, Y_train_lt)
y_pred = DTregressor.predict(X_test_lt)
print('Decision Tree Regressor Long Term:')
Get_score(y_pred, Y_test_lt)

Decision Tree Regressor Long Term:
The Spearman's correlation coefficient is: 0.033


#### Random Forest Regressor Model  on C3D

In [167]:
RFregressor = RandomForestRegressor(n_jobs=-1,n_estimators=100)
RFregressor.fit(X_train_st, Y_train_st)
y_pred = RFregressor.predict(X_test_st)
print('Random Forest Regressor Short Term:')
Get_score(y_pred, Y_test_st)

Random Forest Regressor Short Term:
The Spearman's correlation coefficient is: 0.332


In [168]:
RFregressor = RandomForestRegressor(n_jobs=-1,n_estimators=100)
RFregressor.fit(X_train_lt, Y_train_lt)
y_pred = RFregressor.predict(X_test_lt)
print('Random Forest Regressor Long Term:')
Get_score(y_pred, Y_test_lt)

Random Forest Regressor Long Term:
The Spearman's correlation coefficient is: 0.113


#### XGBoost Regressor Model on C3D

In [86]:
xg_reg = xgb.XGBRegressor(nthread=-1, learning_rate = 1, max_depth = 5, alpha = 10, n_estimators = 100, objective ='reg:squarederror')
xg_reg.fit(X_train_st,Y_train_st)
preds_y = xg_reg.predict(X_test_st)
print('XG Boosting Regressor Short Term:')
Get_score(preds_y, Y_test_st)

XG Boosting Regressor Short Term:
The Spearman's correlation coefficient is: 0.151


In [87]:
xg_reg = xgb.XGBRegressor(nthread=-1, learning_rate = 1, max_depth = 5, alpha = 10, n_estimators = 100, objective ='reg:squarederror')
xg_reg.fit(X_train_lt,Y_train_lt)
preds_y = xg_reg.predict(X_test_lt)
print('XG Boosting Regressor Long Term:')
Get_score(preds_y, Y_test_lt)

XG Boosting Regressor Long Term:
The Spearman's correlation coefficient is: 0.034


#### Gradient Boosting Regressor Model on C3D

In [61]:
clf = ensemble.GradientBoostingRegressor(n_estimators=650, max_depth=12, min_samples_split=2, learning_rate=0.01, loss='lad')

In [62]:
clf.fit(X_train_st, Y_train_st)
print('Gradient Boosting Regressor Short Term:')
print(Get_score(clf.predict(X_test_st), Y_test_st))

Gradient Boosting Regressor Short Term:
The Spearman's correlation coefficient is: 0.320
None


In [63]:
clf.fit(X_train_lt, Y_train_lt)
print('Gradient Boosting Regressor Long Term:')
print(Get_score(clf.predict(X_test_lt), Y_test_lt))

Gradient Boosting Regressor Long Term:
The Spearman's correlation coefficient is: 0.097
None


### Running models on HMP

In [43]:
feat2=feat2.drop('video', axis = 1)

In [88]:
Y_st = labels['short-term_memorability'].values # st targets
Y_lt = labels['long-term_memorability'].values  # lt targets
X = feat2.values # HMP features

In [89]:
X_train_st, X_test_st, Y_train_st, Y_test_st = train_test_split(X,Y_st, test_size=0.2, random_state=42) 
#splitting the short term dev data into a train and validate split of 80 to 20 with a random state for reproducability

X_train_lt, X_test_lt, Y_train_lt, Y_test_lt = train_test_split(X,Y_lt, test_size=0.2, random_state=42) 
# #splitting the long term dev data into a train and validate split of 80 to 20 with a random state for reproducability

In [90]:
#testing to see shape of data split for Short Term
print('X_train', X_train_st.shape)
print('X_test', X_test_st.shape)
print('Y_train', Y_train_st.shape)
print('Y_test', Y_test_st.shape)

X_train (4800, 6075)
X_test (1200, 6075)
Y_train (4800,)
Y_test (1200,)


In [91]:
#testing to see shape of data split for Long Term
print('X_train', X_train_lt.shape)
print('X_test', X_test_lt.shape)
print('Y_train', Y_train_lt.shape)
print('Y_test', Y_test_lt.shape)

X_train (4800, 6075)
X_test (1200, 6075)
Y_train (4800,)
Y_test (1200,)


#### Linear Regressor Model  on HMP

In [69]:
LMregressor = LinearRegression(n_jobs=-1)
LMregressor.fit(X_train_st, Y_train_st)
y_pred = LMregressor.predict(X_test_st)
print('Linear Regressor Short Term:')
Get_score(y_pred, Y_test_st)

Linear Regressor Short Term:
The Spearman's correlation coefficient is: 0.036


In [70]:
LMregressor = LinearRegression(n_jobs=-1)
LMregressor.fit(X_train_lt, Y_train_lt)
y_pred = LMregressor.predict(X_test_lt)
print('Linear Regressor Long Term:')
Get_score(y_pred, Y_test_lt)

Linear Regressor Long Term:
The Spearman's correlation coefficient is: -0.004


#### Decision Tree Regressor Model  on HMP

In [71]:
DTregressor = DecisionTreeRegressor()
DTregressor.fit(X_train_st, Y_train_st)
y_pred = DTregressor.predict(X_test_st)
print('Decision Tree Regressor Short Term:')
Get_score(y_pred, Y_test_st)

Decision Tree Regressor Short Term:
The Spearman's correlation coefficient is: 0.109


In [72]:
DTregressor = DecisionTreeRegressor()
DTregressor.fit(X_train_lt, Y_train_lt)
y_pred = DTregressor.predict(X_test_lt)
print('Decision Tree Regressor Long Term:')
Get_score(y_pred, Y_test_lt)

Decision Tree Regressor Long Term:
The Spearman's correlation coefficient is: 0.042


#### Random Forest Regressor Model on HMP

In [173]:
RFregressor = RandomForestRegressor(n_jobs=-1,n_estimators=100)
RFregressor.fit(X_train_st, Y_train_st)
y_pred = RFregressor.predict(X_test_st)
print('Random Forest Regressor Short Term:')
Get_score(y_pred, Y_test_st)

Random Forest Regressor Short Term:
The Spearman's correlation coefficient is: 0.309


In [174]:
RFregressor = RandomForestRegressor(n_jobs=-1,n_estimators=100)
RFregressor.fit(X_train_lt, Y_train_lt)
y_pred = RFregressor.predict(X_test_lt)
print('Random Forest Regressor Long Term:')
Get_score(y_pred, Y_test_lt)

Random Forest Regressor Long Term:
The Spearman's correlation coefficient is: 0.114


#### XGBoost Regressor Model  on HMP

In [92]:
xg_reg = xgb.XGBRegressor(nthread=-1, learning_rate = 1, max_depth = 5, alpha = 10, n_estimators = 100, objective ='reg:squarederror')
xg_reg.fit(X_train_st,Y_train_st)
preds_y = xg_reg.predict(X_test_st)
print('XG Boosting Regressor Short Term:')
Get_score(preds_y, Y_test_st)

XG Boosting Regressor Short Term:
The Spearman's correlation coefficient is: 0.202


In [93]:
xg_reg = xgb.XGBRegressor(nthread=-1, learning_rate = 1, max_depth = 5, alpha = 10, n_estimators = 100, objective ='reg:squarederror')
xg_reg.fit(X_train_lt,Y_train_lt)
preds_y = xg_reg.predict(X_test_lt)
print('XG Boosting Regressor Long Term:')
Get_score(preds_y, Y_test_lt)

XG Boosting Regressor Long Term:
The Spearman's correlation coefficient is: 0.076


#### Gradient Boosting Regressor Model on HMP

In [137]:
clf = ensemble.GradientBoostingRegressor(n_estimators=650, max_depth=12, min_samples_split=2, learning_rate=0.01, loss='lad')

In [138]:
clf.fit(X_train_st, Y_train_st)
print('Gradient Boosting Regressor Short Term:')
print(Get_score(clf.predict(X_test_st), Y_test_st))

Gradient Boosting Regressor Short Term:
The Spearman's correlation coefficient is: 0.279
None


In [139]:
clf.fit(X_train_lt, Y_train_lt)
print('Gradient Boosting Regressor Long Term:')
print(Get_score(clf.predict(X_test_lt), Y_test_lt))

Gradient Boosting Regressor Long Term:
The Spearman's correlation coefficient is: 0.123
None


### Running models on Captions + C3D

In [62]:
df_x2 = df_x2.drop('video', axis = 1)  #drop video column

In [94]:
Y_st = labels['short-term_memorability'].values # st targets
Y_lt = labels['long-term_memorability'].values  # lt targets
X = df_x2.values # sequences & C3D merged

In [95]:
X_train_st, X_test_st, Y_train_st, Y_test_st = train_test_split(X,Y_st, test_size=0.2, random_state=42) 
#splitting the short term dev data into a train and validate split of 80 to 20 with a random state for reproducability

X_train_lt, X_test_lt, Y_train_lt, Y_test_lt = train_test_split(X,Y_lt, test_size=0.2, random_state=42) 
# #splitting the long term dev data into a train and validate split of 80 to 20 with a random state for reproducability

In [96]:
#Just testing to see shape of data split for Short Term
print('X_train', X_train_st.shape)
print('X_test', X_test_st.shape)
print('Y_train', Y_train_st.shape)
print('Y_test', Y_test_st.shape)

X_train (4800, 151)
X_test (1200, 151)
Y_train (4800,)
Y_test (1200,)


In [97]:
#Just testing to see shape of data split for Long Term
print('X_train', X_train_lt.shape)
print('X_test', X_test_lt.shape)
print('Y_train', Y_train_lt.shape)
print('Y_test', Y_test_lt.shape)

X_train (4800, 151)
X_test (1200, 151)
Y_train (4800,)
Y_test (1200,)


#### Linear Regressor Model on Captions + C3D

In [85]:
LMregressor = LinearRegression(n_jobs=-1)
LMregressor.fit(X_train_st, Y_train_st)
y_pred = LMregressor.predict(X_test_st)
print('Linear Regressor Short Term:')
Get_score(y_pred, Y_test_st)

Linear Regressor Short Term:
The Spearman's correlation coefficient is: 0.273


In [86]:
LMregressor = LinearRegression(n_jobs=-1)
LMregressor.fit(X_train_lt, Y_train_lt)
y_pred = LMregressor.predict(X_test_lt)
print('Linear Regressor Long Term:')
Get_score(y_pred, Y_test_lt)

Linear Regressor Long Term:
The Spearman's correlation coefficient is: 0.081


#### Decision Tree Regressor Model on Captions + C3D

In [87]:
DTregressor = DecisionTreeRegressor()
DTregressor.fit(X_train_st, Y_train_st)
y_pred = DTregressor.predict(X_test_st)
print('Decision Tree Regressor Short Term:')
Get_score(y_pred, Y_test_st)

Decision Tree Regressor Short Term:
The Spearman's correlation coefficient is: 0.086


In [88]:
DTregressor = DecisionTreeRegressor()
DTregressor.fit(X_train_lt, Y_train_lt)
y_pred = DTregressor.predict(X_test_lt)
print('Decision Tree Regressor Long Term:')
Get_score(y_pred, Y_test_lt)

Decision Tree Regressor Long Term:
The Spearman's correlation coefficient is: 0.011


#### Random Forest Regressor Model on Captions + C3D

In [179]:
RFregressor = RandomForestRegressor(n_jobs=-1,n_estimators=100)
RFregressor.fit(X_train_st, Y_train_st)
y_pred = RFregressor.predict(X_test_st)
print('Random Forest Regressor Short Term:')
Get_score(y_pred, Y_test_st)

Random Forest Regressor Short Term:
The Spearman's correlation coefficient is: 0.303


In [180]:
RFregressor = RandomForestRegressor(n_jobs=-1,n_estimators=100)
RFregressor.fit(X_train_lt, Y_train_lt)
y_pred = RFregressor.predict(X_test_lt)
print('Random Forest Regressor Long Term:')
Get_score(y_pred, Y_test_lt)

Random Forest Regressor Long Term:
The Spearman's correlation coefficient is: 0.116


#### XGBoost Regressor Model on Captions + C3D

In [98]:
xg_reg = xgb.XGBRegressor(nthread=-1, learning_rate = 1, max_depth = 5, alpha = 10, n_estimators = 100, objective ='reg:squarederror')
xg_reg.fit(X_train_st,Y_train_st)
preds_y = xg_reg.predict(X_test_st)
print('XG Boosting Regressor Short Term:')
Get_score(preds_y, Y_test_st)

XG Boosting Regressor Short Term:
The Spearman's correlation coefficient is: 0.154


In [100]:
xg_reg = xgb.XGBRegressor(nthread=-1, learning_rate = 1, max_depth = 5, alpha = 10, n_estimators = 100, objective ='reg:squarederror')
xg_reg.fit(X_train_lt,Y_train_lt)
preds_y = xg_reg.predict(X_test_lt)
print('XG Boosting Regressor Long Term:')
Get_score(preds_y, Y_test_lt)

XG Boosting Regressor Long Term:
The Spearman's correlation coefficient is: 0.029


#### Gradient Boosting Regressor Model on Captions + C3D

In [144]:
clf = ensemble.GradientBoostingRegressor(n_estimators=650, max_depth=12, min_samples_split=2, learning_rate=0.01, loss='lad')

In [145]:
clf.fit(X_train_st, Y_train_st)
print('Gradient Boosting Regressor Short Term:')
print(Get_score(clf.predict(X_test_st), Y_test_st))

Gradient Boosting Regressor Short Term:
The Spearman's correlation coefficient is: 0.323
None


In [146]:
clf.fit(X_train_lt, Y_train_lt)
print('Gradient Boosting Regressor Long Term:')
print(Get_score(clf.predict(X_test_lt), Y_test_lt))

Gradient Boosting Regressor Long Term:
The Spearman's correlation coefficient is: 0.104
None


### Running models on Captions + C3D + HMP

In [101]:
Y_st = labels['short-term_memorability'].values # st targets
Y_lt = labels['long-term_memorability'].values  # lt targets
X = df_x.values # sequences & C3D merged

In [102]:
X_train_st, X_test_st, Y_train_st, Y_test_st = train_test_split(X,Y_st, test_size=0.2, random_state=42) 
#splitting the short term dev data into a train and validate split of 80 to 20 with a random state for reproducability

X_train_lt, X_test_lt, Y_train_lt, Y_test_lt = train_test_split(X,Y_lt, test_size=0.2, random_state=42) 
# #splitting the long term dev data into a train and validate split of 80 to 20 with a random state for reproducability

In [103]:
#Just testing to see shape of data split for Short Term
print('X_train', X_train_st.shape)
print('X_test', X_test_st.shape)
print('Y_train', Y_train_st.shape)
print('Y_test', Y_test_st.shape)

X_train (4800, 6226)
X_test (1200, 6226)
Y_train (4800,)
Y_test (1200,)


In [104]:
#Just testing to see shape of data split for Long Term
print('X_train', X_train_lt.shape)
print('X_test', X_test_lt.shape)
print('Y_train', Y_train_lt.shape)
print('Y_test', Y_test_lt.shape)

X_train (4800, 6226)
X_test (1200, 6226)
Y_train (4800,)
Y_test (1200,)


#### Linear Regressor Model on Captions + C3D + HMP

In [97]:
LMregressor = LinearRegression(n_jobs=-1)
LMregressor.fit(X_train_st, Y_train_st)
y_pred = LMregressor.predict(X_test_st)
print('Linear Regressor Short Term:')
Get_score(y_pred, Y_test_st)

Linear Regressor Short Term:
The Spearman's correlation coefficient is: -0.019


In [98]:
LMregressor = LinearRegression(n_jobs=-1)
LMregressor.fit(X_train_lt, Y_train_lt)
y_pred = LMregressor.predict(X_test_lt)
print('Linear Regressor Long Term:')
Get_score(y_pred, Y_test_lt)

Linear Regressor Long Term:
The Spearman's correlation coefficient is: 0.001


#### Decision Tree Regressor Model on Captions + C3D + HMP

In [99]:
DTregressor = DecisionTreeRegressor()
DTregressor.fit(X_train_st, Y_train_st)
y_pred = DTregressor.predict(X_test_st)
print('Decision Tree Regressor Short Term:')
Get_score(y_pred, Y_test_st)

Decision Tree Regressor Short Term:
The Spearman's correlation coefficient is: 0.085


In [100]:
DTregressor = DecisionTreeRegressor()
DTregressor.fit(X_train_lt, Y_train_lt)
y_pred = DTregressor.predict(X_test_lt)
print('Decision Tree Regressor Long Term:')
Get_score(y_pred, Y_test_lt)

Decision Tree Regressor Long Term:
The Spearman's correlation coefficient is: 0.044


#### Random Forest Regressor Model on Captions + C3D + HMP

In [185]:
RFregressor = RandomForestRegressor(n_jobs=-1,n_estimators=100)
RFregressor.fit(X_train_st, Y_train_st)
y_pred = RFregressor.predict(X_test_st)
print('Random Forest Regressor Short Term:')
Get_score(y_pred, Y_test_st)

Random Forest Regressor Short Term:
The Spearman's correlation coefficient is: 0.341


In [186]:
RFregressor = RandomForestRegressor(n_jobs=-1,n_estimators=100)
RFregressor.fit(X_train_lt, Y_train_lt)
y_pred = RFregressor.predict(X_test_lt)
print('Random Forest Regressor Long Term:')
Get_score(y_pred, Y_test_lt)

Random Forest Regressor Long Term:
The Spearman's correlation coefficient is: 0.132


#### XGBoost Regressor Model on Captions + C3D + HMP

In [106]:
xg_reg = xgb.XGBRegressor(nthread=-1, learning_rate = 1, max_depth = 5, alpha = 10, n_estimators = 100, objective ='reg:squarederror')
xg_reg.fit(X_train_st,Y_train_st)
preds_y = xg_reg.predict(X_test_st)
print('XG Boosting Regressor Short Term:')
Get_score(preds_y, Y_test_st)

XG Boosting Regressor Short Term:
The Spearman's correlation coefficient is: 0.170


In [105]:
xg_reg = xgb.XGBRegressor(nthread=-1, learning_rate = 1, max_depth = 5, alpha = 10, n_estimators = 100, objective ='reg:squarederror')
xg_reg.fit(X_train_lt,Y_train_lt)
preds_y = xg_reg.predict(X_test_lt)
print('XG Boosting Regressor Long Term:')
Get_score(preds_y, Y_test_lt)

XG Boosting Regressor Long Term:
The Spearman's correlation coefficient is: 0.075


#### Gradient Boosting Regressor Model on Captions + C3D + HMP

In [151]:
clf = ensemble.GradientBoostingRegressor(n_estimators=650, max_depth=12, min_samples_split=2, learning_rate=0.01, loss='lad')

In [152]:
clf.fit(X_train_st, Y_train_st)
print('Gradient Boosting Regressor Short Term:')
print(Get_score(clf.predict(X_test_st), Y_test_st))

Gradient Boosting Regressor Short Term:
The Spearman's correlation coefficient is: 0.350
None


In [153]:
clf.fit(X_train_lt, Y_train_lt)
print('Gradient Boosting Regressor Long Term:')
print(Get_score(clf.predict(X_test_lt), Y_test_lt))

Gradient Boosting Regressor Long Term:
The Spearman's correlation coefficient is: 0.142
None
